## Agentic retrieval in Azure AI Search

Use this notebook to get started with agentic retrieval in Azure AI Search, which integrates conversation history and large language models (LLMs) on Azure OpenAI to plan, retrieve, and synthesize complex queries.

Steps in this notebook include:

+ Creating an `product` search index.

+ Loading the index with outdoor product from [manuals](./manuals).

+ Creating an products-search-agent in Azure AI Search that points to an LLM for query planning.

Generating answers using the Azure OpenAI client.

## Set up connections

In [99]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True)  # Take environment variables from .env.

endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential, "https://search.azure.com/.default"
)
index_name = "products"
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("GENERATIVE_DEPLOYMENT_NAME", "gpt-4o")
azure_openai_gpt_model = os.getenv("GENERATIVE_DEPLOYMENT_NAME", "gpt-4o")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2025-03-01-preview")
azure_openai_embedding_deployment = os.getenv(
    "EMBEDDING_DEPLOYMENT_NAME", "text-embedding-3-large"
)
azure_openai_embedding_model = os.getenv(
    "EMBEDDING_DEPLOYMENT_NAME", "text-embedding-3-large"
)
agent_name = "products-search-agent"
api_version = "2025-05-01-Preview"
storage_account_url = os.environ["AZURE_STORAGE_ACCOUNT_URL"]
blob_source = "sample-docs-container"
blob_artifacts = "knowledgestore-artifacts"
local_data_source = "manuals"
subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]
resource_group = os.environ["AZURE_RESOURCE_GROUP"]
storage_account_name = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
azure_aiservices_endpoint = os.environ["AZURE_AI_SERVICES_ENDPOINT"]
azure_openai_embeddings_dimensions = 3072

## Create an index in Azure AI Search

In [90]:
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchableField,
    SearchFieldDataType,
    SearchIndex,
    SearchField,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SemanticSearch,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    ComplexField,
    HnswParameters,
)
from azure.search.documents.indexes import SearchIndexClient

fields = [
    SearchableField(
        name="content_id",
        type=SearchFieldDataType.String,
        key=True,
        analyzer_name="keyword",
    ),
    SimpleField(
        name="text_document_id",
        type=SearchFieldDataType.String,
        searchable=False,
        filterable=True,
        hidden=False,
        sortable=False,
        facetable=False,
    ),
    SimpleField(
        name="image_document_id",
        type=SearchFieldDataType.String,
        searchable=False,
        filterable=True,
        hidden=False,
        sortable=False,
        facetable=False,
    ),
    SearchableField(
        name="document_title",
        type=SearchFieldDataType.String,
        searchable=True,
        filterable=True,
        hidden=False,
        sortable=True,
        facetable=True,
    ),
    SearchableField(
        name="content_text",
        type=SearchFieldDataType.String,
        searchable=True,
        filterable=True,
        hidden=False,
        sortable=True,
        facetable=True,
    ),
    SearchField(
        name="content_embedding",
        hidden=False,
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        vector_search_dimensions=azure_openai_embeddings_dimensions,
        searchable=True,
        vector_search_profile_name=f"{index_name}-profile",
    ),
    SimpleField(
        name="content_path",
        type=SearchFieldDataType.String,
        searchable=False,
        filterable=True,
        hidden=False,
        sortable=False,
        facetable=False,
    ),
    ComplexField(
        name="locationMetadata",
        fields=[
            SimpleField(
                name="pageNumber",
                type=SearchFieldDataType.Int32,
                searchable=False,
                filterable=True,
                hidden=False,
                sortable=True,
                facetable=True,
            ),
            SimpleField(
                name="boundingPolygons",
                type=SearchFieldDataType.String,
                searchable=False,
                hidden=False,
                filterable=False,
                sortable=False,
                facetable=False,
            ),
        ],
    ),
]
index = SearchIndex(
    fields=fields,
    name=index_name,
    vector_search=VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name=f"{index_name}-algo",
                parameters=HnswParameters(
                    metric="cosine",
                ),
            )
        ],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name=f"{index_name}-vectorizer",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=azure_openai_embedding_deployment,
                    model_name=azure_openai_embedding_model,
                ),
            )
        ],
        profiles=[
            VectorSearchProfile(
                algorithm_configuration_name=f"{index_name}-algo",
                vectorizer_name=f"{index_name}-vectorizer",
                name=f"{index_name}-profile",
            )
        ],
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semanticconfig",
        configurations=[
            SemanticConfiguration(
                name="semanticconfig",
                prioritized_fields=SemanticPrioritizedFields(
                    title_field=SemanticField(
                        field_name="document_title",
                        weight=1.0,
                    ),
                    content_fields=[
                        SemanticField(
                            field_name="content_text",
                            weight=1.0,
                        )
                    ],
                ),
            )
        ],
    ),
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully")

weight is not a known attribute of class <class 'azure.search.documents.indexes._generated.models._models_py3.SemanticField'> and will be ignored
weight is not a known attribute of class <class 'azure.search.documents.indexes._generated.models._models_py3.SemanticField'> and will be ignored


Index 'products' created or updated successfully


## Create a datasource

In [91]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.core.pipeline.policies import UserAgentPolicy
USER_AGENT = "ai-search-multimodal-sample/1.0.0"

indexer_client = SearchIndexerClient(
    endpoint=endpoint, 
    credential=credential,
    user_agent_policy=UserAgentPolicy(base_user_agent=USER_AGENT)
    )

In [92]:
from azure.storage.blob.aio import BlobServiceClient
import glob
import aiofiles
from azure.search.documents.indexes.models import (
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection,
    SearchIndexerDataSourceType,
    NativeBlobSoftDeleteDeletionDetectionPolicy,
)
import os

blob_service_client = BlobServiceClient(
    account_url=storage_account_url,
    credential=credential,
)

container_client = blob_service_client.get_container_client(blob_source)
try:
    await container_client.create_container()
except Exception as e:
    print(f"Error creating container: {e}")

document_paths = glob.glob(os.path.join(os.getcwd(), local_data_source, "*.*"))
print(f"Document paths: {document_paths}")
for doc_path in document_paths:
    print(f"Uploading file: {doc_path}")
    async with aiofiles.open(doc_path, "rb") as f:
        file_bytes = await f.read()
        file_name = os.path.basename(doc_path)
        await container_client.upload_blob(file_name, file_bytes, overwrite=True)

ds_container = SearchIndexerDataContainer(name=blob_source)
data_source_connection = SearchIndexerDataSourceConnection(
    name=f"{index_name}-blob",
    type=SearchIndexerDataSourceType.AZURE_BLOB,
    connection_string=f"ResourceId=/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.Storage/storageAccounts/{storage_account_name};",
    container=ds_container,
    data_deletion_detection_policy=NativeBlobSoftDeleteDeletionDetectionPolicy(),
)

indexer_client.create_data_source_connection(
        data_source_connection
)

print(f"Data source connection '{data_source_connection.name}' created successfully")

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001C7602D2510>


Error creating container: The specified container already exists.
RequestId:6875b8c6-201e-00f9-30f7-cf0b04000000
Time:2025-05-28T17:40:36.6935717Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:6875b8c6-201e-00f9-30f7-cf0b04000000
Time:2025-05-28T17:40:36.6935717Z</Message></Error>
Document paths: ['c:\\Users\\pablocastao\\OneDrive - Microsoft\\Work\\Clients\\Bancolombia\\gbbai-ai-foundry-sdk-workshop\\2-notebooks\\4-agentic_retrieval_aisearch\\manuals\\product_info_1.pdf', 'c:\\Users\\pablocastao\\OneDrive - Microsoft\\Work\\Clients\\Bancolombia\\gbbai-ai-foundry-sdk-workshop\\2-notebooks\\4-agentic_retrieval_aisearch\\manuals\\product_info_10.pdf', 'c:\\Users\\pablocastao\\OneDrive - Microsoft\\Work\\Clients\\Bancolombia\\gbbai-ai-foundry-sdk-workshop\\2-notebooks\\4-agentic_retrieval_aisearch\\manuals\\product_info_11.pdf', 'c:\\Users\\pablocastao\\On

## Create Skillset

In [93]:
from azure.search.documents.indexes.models import (
    AzureOpenAIEmbeddingSkill,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    DocumentIntelligenceLayoutSkill,
    ShaperSkill,
    ChatCompletionSkill,
    DocumentIntelligenceLayoutSkillChunkingProperties,
)
from datetime import timedelta

# Create document intelligence layout skill
document_layout_skill = DocumentIntelligenceLayoutSkill(
    name="document-cracking-skill",
    description="Document Intelligence skill for document cracking",
    context="/document",
    output_mode="oneToMany",
    output_format="text",
    extraction_options=["images", "locationMetadata"],
    markdown_header_depth="",
    chunking_properties=DocumentIntelligenceLayoutSkillChunkingProperties(
        unit="characters",
        maximum_length=2000,
        overlap_length=200,
    ),
    inputs=[InputFieldMappingEntry(name="file_data", source="/document/file_data")],
    outputs=[
        OutputFieldMappingEntry(name="text_sections", target_name="text_sections"),
        OutputFieldMappingEntry(
            name="normalized_images", target_name="normalized_images"
        ),
    ],
)

# Create Azure OpenAI embedding skill
azure_embedding_skill = AzureOpenAIEmbeddingSkill(
    name="text-embedding-skill",
    context="/document/text_sections/*",
    inputs=[
        InputFieldMappingEntry(name="text", source="/document/text_sections/*/content")
    ],
    outputs=[OutputFieldMappingEntry(name="embedding", target_name="text_vector")],
    resource_url=azure_openai_endpoint,
    deployment_name=azure_openai_embedding_deployment,
    dimensions=azure_openai_embeddings_dimensions,
    model_name=azure_openai_embedding_deployment,
)

# Create gpt skill
gpt_skill = ChatCompletionSkill(
    name="chat-completion-skill",
    uri=f"{azure_openai_endpoint}/openai/deployments/gpt-4.1/chat/completions?api-version=2024-10-21",
    timeout=timedelta(minutes=1),
    context="/document/normalized_images/*",
    inputs=[
        InputFieldMappingEntry(
            name="systemMessage",
            source='=\'You are tasked with generating concise, accurate descriptions of images, figures, diagrams, or charts in documents. The goal is to capture the key information and meaning conveyed by the image without including extraneous details like style, colors, visual aesthetics, or size.\n\nInstructions:\nContent Focus: Describe the core content and relationships depicted in the image.\n\nFor diagrams, specify the main elements and how they are connected or interact.\nFor charts, highlight key data points, trends, comparisons, or conclusions.\nFor figures or technical illustrations, identify the components and their significance.\nClarity & Precision: Use concise language to ensure clarity and technical accuracy. Avoid subjective or interpretive statements.\n\nAvoid Visual Descriptors: Exclude details about:\n\nColors, shading, and visual styles.\nImage size, layout, or decorative elements.\nFonts, borders, and stylistic embellishments.\nContext: If relevant, relate the image to the broader content of the technical document or the topic it supports.\n\nExample Descriptions:\nDiagram: "A flowchart showing the four stages of a machine learning pipeline: data collection, preprocessing, model training, and evaluation, with arrows indicating the sequential flow of tasks."\n\nChart: "A bar chart comparing the performance of four algorithms on three datasets, showing that Algorithm A consistently outperforms the others on Dataset 1."\n\nFigure: "A labeled diagram illustrating the components of a transformer model, including the encoder, decoder, self-attention mechanism, and feedforward layers."\'',
        ),
        InputFieldMappingEntry(
            name="userMessage",
            source="='Please describe this image.'",
        ),
        InputFieldMappingEntry(
            name="image",
            source="/document/normalized_images/*/data",
        ),
    ],
    outputs=[OutputFieldMappingEntry(name="response", target_name="verbalizedImage")],
)


# Create embedding skill for verbalized images
verbalized_skill = AzureOpenAIEmbeddingSkill(
        name="verblizedImage-embedding-skill",
        context="/document/normalized_images/*",
        inputs=[
            InputFieldMappingEntry(
                name="text", source="/document/normalized_images/*/verbalizedImage"
            )
        ],
        outputs=[
            OutputFieldMappingEntry(
                name="embedding", target_name="verbalizedImage_vector"
            )
        ],
        resource_url=azure_openai_endpoint,
        deployment_name=azure_openai_embedding_deployment,
        dimensions=azure_openai_embeddings_dimensions,
        model_name=azure_openai_embedding_deployment,
)

# Create shaper skill to shape the output
shaper_skill = ShaperSkill(
        name="#5",
        context="/document/normalized_images/*",
        inputs=[
            InputFieldMappingEntry(
                name="normalized_images",
                source="/document/normalized_images/*",
                inputs=[],
            ),
            InputFieldMappingEntry(
                name="imagePath",
                source=f"='{blob_artifacts}/'+$(/document/normalized_images/*/imagePath)",
                inputs=[],
            ),
        ],
        outputs=[
            OutputFieldMappingEntry(name="output", target_name="new_normalized_images")
        ],
    )

In [94]:
from azure.search.documents.indexes.models import (
    SearchIndexerSkillset,
    SearchIndexerIndexProjection,
    SearchIndexerIndexProjectionSelector,
    InputFieldMappingEntry,
    AIServicesAccountIdentity,
    SearchIndexerKnowledgeStore,
    SearchIndexerKnowledgeStoreProjection,
    SearchIndexerKnowledgeStoreFileProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    IndexProjectionMode,
)

skillSet = SearchIndexerSkillset(
    name=f"{index_name}-skillset",
    skills=[
        document_layout_skill,
        gpt_skill,
        azure_embedding_skill,
        verbalized_skill,
        shaper_skill,
    ],
    index_projection=SearchIndexerIndexProjection(
        selectors=[
            SearchIndexerIndexProjectionSelector(
                target_index_name=index_name,
                source_context="/document/text_sections/*",
                parent_key_field_name="text_document_id",
                mappings=[
                    InputFieldMappingEntry(
                        name="content_embedding",
                        source="/document/text_sections/*/text_vector",
                    ),
                    InputFieldMappingEntry(
                        name="content_text",
                        source="/document/text_sections/*/content",
                    ),
                    InputFieldMappingEntry(
                        name="locationMetadata",
                        source="/document/text_sections/*/locationMetadata",
                    ),
                    InputFieldMappingEntry(
                        name="document_title", source="/document/document_title"
                    ),
                ],
            ),
            SearchIndexerIndexProjectionSelector(
                target_index_name=index_name,
                source_context="/document/normalized_images/*",
                parent_key_field_name="image_document_id",
                mappings=[
                    InputFieldMappingEntry(
                        name="content_embedding",
                        source="/document/normalized_images/*/verbalizedImage_vector",
                    ),
                    InputFieldMappingEntry(
                        name="content_text",
                        source="/document/normalized_images/*/verbalizedImage",
                    ),
                    InputFieldMappingEntry(
                        name="content_path",
                        source="/document/normalized_images/*/new_normalized_images/imagePath",
                    ),
                    InputFieldMappingEntry(
                        name="locationMetadata",
                        source="/document/normalized_images/*/locationMetadata",
                    ),
                    InputFieldMappingEntry(
                        name="document_title", source="/document/document_title"
                    ),
                ],
            ),
        ],
        parameters=SearchIndexerIndexProjectionsParameters(
            projection_mode=IndexProjectionMode.SKIP_INDEXING_PARENT_DOCUMENTS
        ),
    ),
    cognitive_services_account=AIServicesAccountIdentity(
        subdomain_url=azure_aiservices_endpoint,
    ),
    knowledge_store=SearchIndexerKnowledgeStore(
        storage_connection_string=f"ResourceId=/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.Storage/storageAccounts/{storage_account_name};",
        projections=[
            SearchIndexerKnowledgeStoreProjection(
                files=[
                    SearchIndexerKnowledgeStoreFileProjectionSelector(
                        storage_container=blob_artifacts,
                        source="/document/normalized_images/*",
                    )
                ]
            )
        ],
    ),
)

indexer_client.create_or_update_skillset(skillSet)
print(f"Skillset '{skillSet.name}' created successfully")

Skillset 'products-skillset' created successfully


## Create Indexer

In [95]:
from azure.search.documents.indexes.models import (
    SearchIndexer,
    IndexingParameters,
    IndexingParametersConfiguration,
    FieldMapping
)

search_indexer = indexer_client.create_or_update_indexer(
            indexer=SearchIndexer(
                name=f"{index_name}-indexer",
                data_source_name=f"{index_name}-blob",
                target_index_name=f"{index_name}",
                skillset_name=f"{index_name}-skillset",
                parameters=IndexingParameters(
                    batch_size=1,
                    configuration=IndexingParametersConfiguration(
                        data_to_extract="contentAndMetadata",
                        allow_skillset_to_read_file_data=True,
                        query_timeout=None,
                    ),
                ),
                field_mappings=[
                    FieldMapping(
                        source_field_name="metadata_storage_name",
                        target_field_name="document_title",
                    ),
                ],
            )
        )

indexer_client.run_indexer(search_indexer.name)
print(f"Indexer '{search_indexer.name}' created and started successfully")

Indexer 'products-indexer' created and started successfully


## Classic Search

In [ ]:
from azure.search.documents.models import VectorizableTextQuery
import json 

credential = DefaultAzureCredential()

index_client = SearchIndexClient(
    endpoint=endpoint,
    credential=credential
)

index = index_client.get_search_client(index_name)
query = "Which accomodates more people TrailMaster or SkyView?"
results = index.search(
    search_text = query,
    top=2,
    query_type="semantic",
    vector_queries=[
        VectorizableTextQuery(
            text=query,
            k_nearest_neighbors=50,
            fields="content_embedding"
        )
    ]
)

output_list = []
for result in results:
    output_list.append({
        "document_title": result.get("document_title"),
        "content_text": result.get("content_text"),
    })
print(json.dumps(output_list, indent=2))

[
  {
    "document_title": "product_info_1.pdf",
    "content_text": "# Information about product item_number: 1TrailMaster X4 Tent, price $250,## BrandOutdoorLiving## Category Tents## Features- Polyester material for durability- Spacious interior to accommodate multiple people- Easy setup with included instructions- Water-resistant construction to withstand light rain- Mesh panels for ventilation and insect protection- Rainfly included for added weather protection- Multiple doors for convenient entry and exit- Interior pockets for organizing small items- Reflective guy lines for improved visibility at night- Freestanding design for easy setup and relocation - Carry bag included for convenient storage and transportation## Technical Specs ** Best Use **: Camping ** Capacity **: 4-person** Season Rating **: 3-season ** Setup **: Freestanding"
  },
  {
    "document_title": "product_info_1.pdf",
    "content_text": "anyone who loves camping!2) ** Rating :** 3** Review :** I bought the Tr

## Agentic Search

In [130]:
from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeAgentTargetIndex, KnowledgeAgentRequestLimits, AzureOpenAIVectorizerParameters
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, KnowledgeAgentIndexParams
import json
import textwrap

credential = DefaultAzureCredential()

In [ ]:
# 1. Create an Agent
agent = KnowledgeAgent(
    name=agent_name,
    models=[
        KnowledgeAgentAzureOpenAIModel(
            azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
                resource_url=azure_openai_endpoint,
                deployment_name=azure_openai_gpt_deployment,
                model_name=azure_openai_gpt_model
            )
        )
    ],
    target_indexes=[
        KnowledgeAgentTargetIndex(
            index_name=index_name,
            default_reranker_threshold=2.5
        )
    ],
)

index_client.create_or_update_agent(agent)

In [ ]:
# 2. Create a Retrieval Client
agent_client = KnowledgeAgentRetrievalClient(endpoint=endpoint, agent_name=agent_name, credential=credential)

messages = [
    KnowledgeAgentMessage(
        role="user", 
        content=[
            KnowledgeAgentMessageTextContent(text="What are examples of popular tents?")
        ]
    ),
    KnowledgeAgentMessage(
        role="assistant",
        content=[
            KnowledgeAgentMessageTextContent(text="TrailMaster and SkyView are two popular ones")
        ]
    ),
    KnowledgeAgentMessage(
        role="user",
        content=[
            KnowledgeAgentMessageTextContent(text="Which one fits more people?")
        ]
    )
]

retrieval_result = agent_client.retrieve(
    retrieval_request=KnowledgeAgentRetrievalRequest(
        messages=messages,
        target_index_params=[KnowledgeAgentIndexParams(index_name=index_name, reranker_threshold=2.5)]
    )
)

text = retrieval_result.response[0].content[0].text
print("Response:\n")
print(textwrap.fill(text, width=120))

Response:

[{"ref_id":0,"title":"product_info_1.pdf","content":"# Information about product item_number: 1TrailMaster X4 Tent,
price $250,## BrandOutdoorLiving## Category Tents## Features- Polyester material for durability- Spacious interior to
accommodate multiple people- Easy setup with included instructions- Water-resistant construction to withstand light
rain- Mesh panels for ventilation and insect protection- Rainfly included for added weather protection- Multiple doors
for convenient entry and exit- Interior pockets for organizing small items- Reflective guy lines for improved visibility
at night- Freestanding design for easy setup and relocation - Carry bag included for convenient storage and
transportation## Technical Specs ** Best Use **: Camping ** Capacity **: 4-person** Season Rating **: 3-season ** Setup
**: Freestanding"},{"ref_id":1,"title":"product_info_15.pdf","content":"- ** Capacity **: 2-person- ** Seasons **:
3-season - ** Packed Weight **: Approx. 8 lbs - ** Numbe

In [114]:
print("Activity")
print(json.dumps([a.as_dict() for a in retrieval_result.activity], indent=2))

Activity
[
  {
    "id": 0,
    "type": "ModelQueryPlanning",
    "input_tokens": 1249,
    "output_tokens": 200
  },
  {
    "id": 1,
    "type": "AzureSearchQuery",
    "target_index": "products",
    "query": {
      "search": "TrailMaster tent maximum capacity"
    },
    "query_time": "2025-05-28T18:11:45.596Z",
    "count": 2,
    "elapsed_ms": 266
  },
  {
    "id": 2,
    "type": "AzureSearchQuery",
    "target_index": "products",
    "query": {
      "search": "SkyView tent maximum capacity"
    },
    "query_time": "2025-05-28T18:11:45.815Z",
    "count": 2,
    "elapsed_ms": 209
  },
  {
    "id": 3,
    "type": "AzureSearchSemanticRanker",
    "input_tokens": 37432
  }
]
